# GMM Health Phenotype Discovery

## MSc Public Health Data Science - SDS6217 Advanced Machine Learning

---

**Group 6 Members:**

| Student ID            | Student Name          |
|-----------------------|-----------------------|
| SDS6/46982/2024       | Cavin Otieno          |
| SDS6/46284/2024       | Joseph Ongoro Marindi |
| SDS6/47543/2024       | Laura Nabalayo Kundu  |
| SDS6/47545/2024       | Nevin Khaemba         |

---

**Date:** January 2025  
**Institution:** University of Nairobi  

---

### Project Overview

This comprehensive data science project applies Gaussian Mixture Models (GMM) to identify latent subpopulations in NHANES health data, demonstrating how probabilistic clustering can capture population heterogeneity that traditional hard-clustering methods may miss.

### Dataset

**Source:** National Health and Nutrition Examination Survey (NHANES)  
**Location:** `data/raw/nhanes_health_data.csv`  
**Samples:** 5,000 respondents  
**Variables:** 47 health indicators

### Key Features

- **Probabilistic Clustering**: Captures uncertainty in cluster assignments  
- **Hyperparameter Tuning**: Systematic grid search optimization  
- **Population Phenotype Discovery**: Identifies distinct health subgroups  
- **Academic Rigor**: Comprehensive methodology suitable for MSc-level assessment

### Why GMM for Public Health?

1. **Probabilistic Cluster Assignment**: Unlike K-Means which forces hard assignments, GMM provides posterior probabilities. Each individual receives a probability of belonging to each cluster, which is critical for health decisions where uncertainty quantification matters.

2. **Modeling Population Heterogeneity**: Health populations naturally exhibit continuous distributions of risk factors. GMM captures latent subgroups without imposing artificial boundaries, reflecting the biological reality of disease processes.

3. **Flexibility Through Covariance Structures**: Four covariance types allow modeling of various cluster shapes. Full covariance captures elongated, correlated clusters, while diagonal and spherical options provide computational efficiency.

4. **Uncertainty Quantification**: Confidence in cluster assignments can be assessed, which is important for clinical decision-making and resource allocation.

In [ ]:
"""
================================================================================
PROJECT CONFIGURATION - EMBEDDED PATHS AND UTILITIES
================================================================================

This module provides centralized project configuration, path management, and 
utility functions for the GMM Health Phenotype Discovery project.

# Author: Cavin Otieno
# Student ID: SDS6/46982/2024
# Group 6 - Team Project
MSc Public Health Data Science - SDS6217 Advanced Machine Learning
University of Nairobi
"""

import os
import sys
import joblib
import json
import warnings
from datetime import datetime

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')

# =============================================================================
# PROJECT ROOT AND DIRECTORY PATHS
# =============================================================================

# Define project root directory (current working directory)
PROJECT_ROOT = os.path.abspath(os.path.dirname('__file__'))

# Define main directory paths
DATA_DIR = os.path.join(PROJECT_ROOT, 'data')
OUTPUT_DIR = os.path.join(PROJECT_ROOT, 'output_v2')
MODELS_DIR = os.path.join(PROJECT_ROOT, 'models')
FIGURES_DIR = os.path.join(PROJECT_ROOT, 'figures')

# Define phase-specific subdirectories
PHASE_DIRS = {
    'data': os.path.join(DATA_DIR, 'raw'),
    'processed': os.path.join(DATA_DIR, 'processed'),
    'reports': os.path.join(OUTPUT_DIR, 'reports'),
    'logs': os.path.join(OUTPUT_DIR, 'logs'),
    'plots': os.path.join(FIGURES_DIR, 'plots')
}

# Define model subdirectories
MODEL_SUBDIRS = {
    'gmm_clustering': os.path.join(MODELS_DIR, 'gmm_clustering'),
    'baseline': os.path.join(MODELS_DIR, 'baseline'),
    'tuned': os.path.join(MODELS_DIR, 'tuned'),
    'final': os.path.join(MODELS_DIR, 'final'),
    'comparison': os.path.join(MODELS_DIR, 'comparison')
}

# Define output subdirectories
OUTPUT_SUBDIRS = {
    'metrics': os.path.join(OUTPUT_DIR, 'metrics'),
    'predictions': os.path.join(OUTPUT_DIR, 'predictions'),
    'thresholds': os.path.join(OUTPUT_DIR, 'thresholds'),
    'fairness': os.path.join(OUTPUT_DIR, 'fairness'),
    'validation': os.path.join(OUTPUT_DIR, 'validation'),
    'cluster_profiles': os.path.join(OUTPUT_DIR, 'cluster_profiles')
}

# =============================================================================
# UTILITY FUNCTIONS
# =============================================================================

def create_directory_structure():
    """Create all project directories if they don't exist."""
    all_dirs = [
        PROJECT_ROOT, DATA_DIR, OUTPUT_DIR, MODELS_DIR, FIGURES_DIR,
        *PHASE_DIRS.values(), *MODEL_SUBDIRS.values(), *OUTPUT_SUBDIRS.values()
    ]
    created = []
    for dir_path in all_dirs:
        if dir_path and not os.path.exists(dir_path):
            os.makedirs(dir_path, exist_ok=True)
            created.append(dir_path)
    if created:
        print(f"Created {len(created)} directory(ies)")
    return created

def save_fig(figure, filename, subdir=None, formats=['png', 'pdf', 'svg']):
    """Save a matplotlib figure in multiple formats."""
    save_dir = FIGURES_DIR
    if subdir:
        save_dir = os.path.join(FIGURES_DIR, subdir)
        os.makedirs(save_dir, exist_ok=True)
    saved_files = []
    for fmt in formats:
        filepath = os.path.join(save_dir, f"{filename}.{fmt}")
        figure.savefig(filepath, dpi=300, bbox_inches='tight', format=fmt)
        saved_files.append(filepath)
    return saved_files

def save_fig_multi_format(filename, figure=None, subdir=None,
                          dpi=300, bbox_inches='tight',
                          formats=['png', 'pdf', 'svg']):
    """Save figure in multiple formats with consistent naming."""
    if figure is None:
        figure = plt.gcf()
    save_dir = FIGURES_DIR
    if subdir:
        save_dir = os.path.join(FIGURES_DIR, subdir)
        os.makedirs(save_dir, exist_ok=True)
    saved = []
    for fmt in formats:
        filepath = os.path.join(save_dir, f"{filename}.{fmt}")
        figure.savefig(filepath, dpi=dpi, bbox_inches=bbox_inches, format=fmt)
        saved.append(filepath)
    return saved

def save_model(model, filename, subdir=None, model_type=None):
    """Save a trained model using joblib.
    
    Parameters:
    -----------
    model : sklearn model
        The trained model to save
    filename : str
        The filename for the model (without extension)
    subdir : str, optional
        Subdirectory within MODELS_DIR to save to
    model_type : str, optional
        Alias for subdir - category of model (e.g., 'tuned', 'final')
    """
    directory = subdir if subdir else model_type
    if directory and directory in MODEL_SUBDIRS:
        save_dir = MODEL_SUBDIRS[directory]
    else:
        save_dir = MODELS_DIR
    os.makedirs(save_dir, exist_ok=True)
    filepath = os.path.join(save_dir, f"{filename}.joblib")
    joblib.dump(model, filepath)
    return filepath

def load_model(filepath):
    """Load a trained model using joblib."""
    return joblib.load(filepath)

def save_data(data, filename, subdir=None, fmt='csv'):
    """Save data (DataFrame or array) to file."""
    if subdir and subdir in OUTPUT_SUBDIRS:
        save_dir = OUTPUT_SUBDIRS[subdir]
    else:
        save_dir = OUTPUT_DIR
    os.makedirs(save_dir, exist_ok=True)
    filepath = os.path.join(save_dir, f"{filename}.{fmt}")
    if fmt == 'csv':
        if hasattr(data, 'to_csv'):
            data.to_csv(filepath, index=False)
        else:
            pd.DataFrame(data).to_csv(filepath, index=False)
    elif fmt == 'json':
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=2)
    return filepath

def load_data(filepath):
    """Load data from file."""
    if filepath.endswith('.csv'):
        return pd.read_csv(filepath)
    elif filepath.endswith('.json'):
        with open(filepath, 'r') as f:
            return json.load(f)
    else:
        raise ValueError(f"Unsupported file format: {filepath}")

def get_data_path(filename):
    """Get the full path to a data file in the raw data directory."""
    return os.path.join(PHASE_DIRS['data'], filename)

def display_configuration():
    """Display current project configuration."""
    config = {
        'Project Root': PROJECT_ROOT,
        'Data Directory': DATA_DIR,
        'Output Directory': OUTPUT_DIR,
        'Models Directory': MODELS_DIR,
        'Figures Directory': FIGURES_DIR,
        'Raw Data Path': PHASE_DIRS['data'],
        'Processed Data Path': PHASE_DIRS['processed'],
        'Timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    }
    
    print("=" * 60)
    print("PROJECT CONFIGURATION")
    print("=" * 60)
    for key, value in config.items():
        print(f"  {key}: {value}")
    print("=" * 60)

# Create all directories on import
created_dirs = create_directory_structure()

# Display configuration
display_configuration()

print("\nProject configuration loaded successfully!")
print(f"Raw data directory: {PHASE_DIRS['data']}")
print(f"Output directory: {OUTPUT_DIR}")
print(f"Models directory: {MODELS_DIR}")
print(f"Figures directory: {FIGURES_DIR}")

In [ ]:
# =============================================================================
# PROJECT PHASES OVERVIEW
# =============================================================================

# =============================================================================
# LITERATURE REVIEW AND ACADEMIC JUSTIFICATION
# =============================================================================

print("=" * 70)
print("LITERATURE REVIEW: WHY GMM FOR PUBLIC HEALTH?")
print("=" * 70)

print("""
Theoretical Foundation
----------------------
Gaussian Mixture Models (GMM) represent a sophisticated probabilistic approach to
clustering that overcomes significant limitations of traditional methods like K-Means.
In public health research, where population heterogeneity is the norm rather than
the exception, GMM's ability to model overlapping subpopulations with probabilistic
membership estimates makes it particularly suitable for complex health data.

Key Advantages for Public Health Applications
----------------------------------------------

1. Probabilistic Cluster Assignment
   - Unlike K-Means which forces hard assignments, GMM provides posterior probabilities
   - Each individual receives a probability of belonging to each cluster
   - Critical for health decisions where uncertainty quantification matters

2. Modeling Population Heterogeneity
   - Health populations naturally exhibit continuous distributions of risk factors
   - GMM captures latent subgroups without imposing artificial boundaries
   - Reflects the biological reality of disease processes

3. Flexibility Through Covariance Structures
   - Four covariance types allow modeling of various cluster shapes
   - Full covariance captures elongated, correlated clusters
   - Diagonal and spherical options provide computational efficiency

4. Uncertainty Quantification
   - Confidence in cluster assignments can be assessed
   - Important for clinical decision-making and resource allocation

Academic Justification Statement
--------------------------------
> Health populations rarely form hard, discrete clusters. Gaussian Mixture Models
> provide a statistically principled approach to capturing latent subpopulations
> with associated uncertainty, making them ideal for public health research.
""")

print("=" * 70)
print("PROJECT PHASES")
print("=" * 70)

phases = [
    ("Phase 1", "Library Imports and Environment Setup", "Import required packages"),
    ("Phase 2", "Data Acquisition - NHANES Dataset Loading", "Load and validate data"),
    ("Phase 3", "Exploratory Data Analysis", "Explore distributions"),
    ("Phase 4", "Data Preprocessing for GMM", "Handle missing values"),
    ("Phase 5", "Dimensionality Reduction", "PCA and t-SNE"),
    ("Phase 6", "GMM Hyperparameter Tuning", "BIC/AIC optimization"),
    ("Phase 7", "Train Optimal GMM Model", "Fit final model"),
    ("Phase 8", "Cluster Interpretation", "Profile subpopulations"),
    ("Phase 9", "Cluster Visualization", "2D/3D plots"),
    ("Phase 10", "Model Evaluation", "Evaluate clustering"),
    ("Phase 11", "Probabilistic Membership Analysis", "Posterior probabilities"),
    ("Phase 12", "Conclusions and Future Work", "Summary and recommendations")
]

for phase, title, desc in phases:
    print(f"{phase}: {title}")
    print(f"   {desc}\n")

print("=" * 70)
print("TOTAL: 12 Phases")
print("=" * 70)


In [ ]:
# =============================================================================
# PHASE 3: EXPLORATORY DATA ANALYSIS
# =============================================================================

print("=" * 70)
print("EXPLORATORY DATA ANALYSIS")
print("=" * 70)

# Define feature categories for analysis
DEMOGRAPHIC_VARS = ['sex', 'age', 'race_ethnicity', 'education_level', 'income_category']
BODY_MEASURE_VARS = ['weight_kg', 'height_cm', 'bmi', 'waist_circumference_cm']
BLOOD_PRESSURE_VARS = ['systolic_bp_mmHg', 'diastolic_bp_mmHg']
LABORATORY_VARS = ['total_cholesterol_mg_dL', 'hdl_cholesterol_mg_dL', 'ldl_cholesterol_mg_dL',
                   'fasting_glucose_mg_dL', 'insulin_uU_mL']
BEHAVIORAL_VARS = ['smoked_100_cigarettes', 'cigarettes_per_day', 'alcohol_use_past_year',
                   'drinks_per_week', 'vigorous_work_activity', 'moderate_work_activity',
                   'vigorous_recreation_activity', 'moderate_recreation_activity']
MEDICAL_VARS = ['general_health_rating', 'arthritis', 'heart_failure', 'coronary_heart_disease',
               'angina_pectoris', 'heart_attack', 'stroke', 'cancer_diagnosis']
MENTAL_HEALTH_VARS = ['phq9_little_interest', 'phq9_feeling_down', 'phq9_sleep_trouble',
                      'phq9_feeling_tired', 'phq9_poor_appetite', 'phq9_feeling_bad_about_self',
                      'phq9_trouble_concentrating', 'phq9_moving_speaking', 'phq9_suicidal_thoughts',
                      'phq9_total_score']

# Summary statistics for key continuous variables
CONTINUOUS_VARS = BODY_MEASURE_VARS + BLOOD_PRESSURE_VARS + LABORATORY_VARS + ['age', 'phq9_total_score']

print("\n" + "-" * 70)
print("SUMMARY STATISTICS FOR KEY HEALTH INDICATORS")
print("-" * 70)

summary_stats = data[CONTINUOUS_VARS].describe().T
summary_stats['median'] = data[CONTINUOUS_VARS].median()
summary_stats['skewness'] = data[CONTINUOUS_VARS].skew()
summary_stats['kurtosis'] = data[CONTINUOUS_VARS].kurtosis()

print("\nDescriptive Statistics:")
display(summary_stats.round(2))

# Create distribution plots
fig, axes = plt.subplots(4, 4, figsize=(16, 14))
axes = axes.flatten()

for i, col in enumerate(CONTINUOUS_VARS):
    ax = axes[i]
    ax.hist(data[col].dropna(), bins=30, color='steelblue', edgecolor='white', alpha=0.7)
    ax.axvline(data[col].mean(), color='red', linestyle='--', linewidth=2, label=f'Mean: {data[col].mean():.1f}')
    ax.axvline(data[col].median(), color='green', linestyle='-', linewidth=2, label=f'Median: {data[col].median():.1f}')
    ax.set_title(f'{col}', fontsize=10, fontweight='bold')
    ax.set_xlabel('')
    ax.legend(fontsize=8)

# Remove extra subplots
for j in range(len(CONTINUOUS_VARS), len(axes)):
    fig.delaxes(axes[j])

plt.suptitle('Distribution of Key Health Indicators (NHANES)', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
os.makedirs(os.path.join(FIGURES_DIR, 'plots'), exist_ok=True)
plt.savefig(os.path.join(FIGURES_DIR, 'plots', '01_health_indicator_distributions.png'), 
            dpi=150, bbox_inches='tight')
plt.show()
print("\n[OK] Figure saved: figures/plots/01_health_indicator_distributions.png")

In [ ]:
# Correlation heatmap for health indicators
print("\n" + "-" * 70)
print("CORRELATION ANALYSIS")
print("-" * 70)

# Select key continuous variables for correlation
corr_vars = ['bmi', 'systolic_bp_mmHg', 'diastolic_bp_mmHg', 
             'total_cholesterol_mg_dL', 'hdl_cholesterol_mg_dL', 
             'fasting_glucose_mg_dL', 'age', 'phq9_total_score']

fig, ax = plt.subplots(figsize=(12, 10))
correlation_matrix = data[corr_vars].corr()

# Create heatmap
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, 
            mask=mask,
            annot=True, 
            fmt='.2f', 
            cmap='RdBu_r', 
            center=0,
            square=True,
            linewidths=0.5,
            cbar_kws={'shrink': 0.8, 'label': 'Correlation Coefficient'},
            ax=ax)

ax.set_title('Correlation Matrix of Health Indicators (NHANES)', fontsize=14, fontweight='bold', pad=20)
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, 'plots', '02_correlation_heatmap.png'), 
            dpi=150, bbox_inches='tight')
plt.show()
print("\n[OK] Figure saved: figures/plots/02_correlation_heatmap.png")

In [ ]:
# =============================================================================
# PHASE 4: DATA PREPROCESSING FOR GMM
# =============================================================================

from sklearn.preprocessing import StandardScaler

print("=" * 70)
print("DATA PREPROCESSING FOR GMM CLUSTERING")
print("=" * 70)

# Define features for GMM clustering
# Select continuous health indicators that are relevant for phenotype discovery
FEATURE_COLUMNS = [
    # Demographics
    'age',
    
    # Body Measures
    'bmi',
    'waist_circumference_cm',
    
    # Blood Pressure
    'systolic_bp_mmHg',
    'diastolic_bp_mmHg',
    
    # Cholesterol
    'total_cholesterol_mg_dL',
    'hdl_cholesterol_mg_dL',
    'ldl_cholesterol_mg_dL',
    
    # Glucose Metabolism
    'fasting_glucose_mg_dL',
    'insulin_uU_mL',
    
    # Mental Health
    'phq9_total_score'
]

print(f"\n[INFO] Selected {len(FEATURE_COLUMNS)} features for GMM clustering:")
for i, col in enumerate(FEATURE_COLUMNS, 1):
    print(f"  {i:2d}. {col}")

# Create feature matrix
X = data[FEATURE_COLUMNS].copy()

# Check for missing values
missing_summary = X.isnull().sum()
if missing_summary.sum() > 0:
    print(f"\n[WARNING] Missing values detected:")
    for col in FEATURE_COLUMNS:
        if X[col].isnull().sum() > 0:
            print(f"  {col}: {X[col].isnull().sum()} ({100*X[col].isnull().sum()/len(X):.1f}%)")
    
    # Impute missing values with median
    X = X.fillna(X.median())
    print("\n[OK] Missing values imputed with median values")
else:
    print("\n[OK] No missing values in selected features")

# Apply Standard Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=FEATURE_COLUMNS)

print(f"\n[OK] Feature scaling applied using StandardScaler")
print(f"  Original data shape: {X.shape}")
print(f"  Scaled data shape: {X_scaled.shape}")

# Save scaler for future use
os.makedirs(os.path.join(MODELS_DIR, 'gmm_clustering'), exist_ok=True)
scaler_path = os.path.join(MODELS_DIR, 'gmm_clustering', 'standard_scaler.joblib')
joblib.dump(scaler, scaler_path)
print(f"\n[OK] Scaler saved to: {scaler_path}")

# Display scaled data summary
print("\nScaled Data Summary:")
print("-" * 70)
scaled_summary = pd.DataFrame(X_scaled, columns=FEATURE_COLUMNS).describe().T
display(scaled_summary[['mean', 'std', 'min', 'max']].round(4))

In [ ]:
# =============================================================================
# PHASE 5: DIMENSIONALITY REDUCTION FOR VISUALIZATION
# =============================================================================

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

print("=" * 70)
print("DIMENSIONALITY REDUCTION FOR VISUALIZATION")
print("=" * 70)

# PCA for visualization
print("\n[INFO] Applying Principal Component Analysis (PCA)...")
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)

print(f"  Explained variance ratio: PC1={pca.explained_variance_ratio_[0]:.3f}, PC2={pca.explained_variance_ratio_[1]:.3f}")
print(f"  Total explained variance: {sum(pca.explained_variance_ratio_):.3f}")

# t-SNE for visualization
print("\n[INFO] Applying t-SNE for nonlinear dimensionality reduction...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
X_tsne = tsne.fit_transform(X_scaled)
print("  t-SNE completed with perplexity=30")

# Visualize PCA and t-SNE projections
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# PCA
scatter1 = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], 
                           c=data['bmi'], cmap='viridis',
                           alpha=0.6, s=30, edgecolor='white', linewidth=0.3)
axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
axes[0].set_title('PCA Projection of NHANES Health Data\n(Color: BMI)', fontweight='bold')
plt.colorbar(scatter1, ax=axes[0], label='BMI')

# t-SNE
scatter2 = axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], 
                           c=data['bmi'], cmap='viridis',
                           alpha=0.6, s=30, edgecolor='white', linewidth=0.3)
axes[1].set_xlabel('t-SNE Dimension 1')
axes[1].set_ylabel('t-SNE Dimension 2')
axes[1].set_title('t-SNE Projection of NHANES Health Data\n(Color: BMI)', fontweight='bold')
plt.colorbar(scatter2, ax=axes[1], label='BMI')

plt.suptitle('Dimensionality Reduction of NHANES Health Indicators', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, 'plots', '03_dimensionality_reduction.png'), 
            dpi=150, bbox_inches='tight')
plt.show()
print("\n[OK] Figure saved: figures/plots/03_dimensionality_reduction.png")

In [ ]:
# =============================================================================
# PHASE 6: GAUSSIAN MIXTURE MODELS - HYPERPARAMETER TUNING
# =============================================================================

from sklearn.mixture import GaussianMixture
from sklearn.model_selection import train_test_split
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

print("=" * 70)
print("GAUSSIAN MIXTURE MODELS - HYPERPARAMETER TUNING")
print("=" * 70)

# Split data for model validation
X_train, X_test = train_test_split(X_scaled, test_size=0.2, random_state=42)

print(f"\n[INFO] Data Split:")
print(f"  Training set: {X_train.shape[0]} samples")
print(f"  Test set: {X_test.shape[0]} samples")

# Define hyperparameter grid
param_grid = {
    'n_components': [2, 3, 4, 5, 6],
    'covariance_type': ['full', 'tied', 'diag', 'spherical'],
    'n_init': [5, 10],
    'reg_covar': [1e-6, 1e-5]
}

print(f"\n[INFO] Hyperparameter Grid:")
print(f"  n_components: {param_grid['n_components']}")
print(f"  covariance_type: {param_grid['covariance_type']}")
print(f"  n_init: {param_grid['n_init']}")
print(f"  reg_covar: {param_grid['reg_covar']}")

total_combinations = (len(param_grid['n_components']) * len(param_grid['covariance_type']) * 
                      len(param_grid['n_init']) * len(param_grid['reg_covar']))
print(f"\n  Total combinations: {total_combinations}")

from itertools import product

def run_grid_search_gmm(X, param_grid):
    """Perform exhaustive grid search for GMM hyperparameters."""
    results = []
    
    # Generate all combinations
    keys = list(param_grid.keys())
    values = [param_grid[k] for k in keys]
    combinations = [dict(zip(keys, v)) for v in product(*values)]
    
    total = len(combinations)
    print(f"\n[INFO] Evaluating {total} model configurations...")
    
    for i, params in enumerate(combinations):
        try:
            gmm = GaussianMixture(
                n_components=params['n_components'],
                covariance_type=params['covariance_type'],
                n_init=params['n_init'],
                reg_covar=params['reg_covar'],
                random_state=42,
                max_iter=200
            )
            
            gmm.fit(X)
            labels = gmm.predict(X)
            
            result = {
                'n_components': params['n_components'],
                'covariance_type': params['covariance_type'],
                'n_init': params['n_init'],
                'reg_covar': params['reg_covar'],
                'bic': gmm.bic(X),
                'aic': gmm.aic(X),
                'log_likelihood': gmm.score(X),
                'silhouette': silhouette_score(X, labels),
                'calinski_harabasz': calinski_harabasz_score(X, labels),
                'davies_bouldin': davies_bouldin_score(X, labels),
                'converged': gmm.converged_
            }
            
            results.append(result)
            
            if (i + 1) % 20 == 0 or i == 0:
                print(f"    Progress: {i+1}/{total} ({100*(i+1)/total:.1f}%)")
                
        except Exception as e:
            print(f"    Error with parameters {params}: {e}")
            continue
    
    return pd.DataFrame(results)

# Run grid search
print("\n[INFO] Running Grid Search with BIC optimization...")
grid_results = run_grid_search_gmm(X_train, param_grid)

# Sort by BIC to find best model
grid_results_sorted = grid_results.sort_values('bic').reset_index(drop=True)

print("\n" + "-" * 70)
print("TOP 10 MODELS BY BIC (Best to Worst)")
print("-" * 70)

top_models = grid_results_sorted.head(10)[['n_components', 'covariance_type', 'n_init', 
                                            'bic', 'aic', 'silhouette', 'converged']]
display(top_models(index=False))

In [ ]:
# =============================================================================
# PHASE 7: TRAIN OPTIMAL GMM MODEL
# =============================================================================

print("=" * 70)
print("TRAINING OPTIMAL GMM MODEL")
print("=" * 70)

# Get best parameters
best_idx = grid_results_sorted.index[0]
best_params = {
    'n_components': int(grid_results_sorted.loc[best_idx, 'n_components']),
    'covariance_type': grid_results_sorted.loc[best_idx, 'covariance_type'],
    'n_init': int(grid_results_sorted.loc[best_idx, 'n_init']),
    'reg_covar': grid_results_sorted.loc[best_idx, 'reg_covar']
}

print(f"\n[OK] BEST MODEL CONFIGURATION:")
print("-" * 50)
for param, value in best_params.items():
    print(f"  {param}: {value}")
print(f"\n  BIC Score: {grid_results_sorted.loc[best_idx, 'bic']:.2f}")
print(f"  AIC Score: {grid_results_sorted.loc[best_idx, 'aic']:.2f}")
print(f"  Silhouette Score: {grid_results_sorted.loc[best_idx, 'silhouette']:.4f}")

# Train the optimal model
gmm_optimal = GaussianMixture(
    n_components=best_params['n_components'],
    covariance_type=best_params['covariance_type'],
    n_init=best_params['n_init'],
    reg_covar=best_params['reg_covar'],
    random_state=42,
    max_iter=500
)

gmm_optimal.fit(X_train)

print(f"\n[OK] Optimal GMM Model Trained Successfully")
print(f"  Convergence: {gmm_optimal.converged_}")
print(f"  Number of iterations: {gmm_optimal.n_iter_}")

# Save the optimal model
model_filepath = os.path.join(MODELS_DIR, 'gmm_clustering', 'gmm_optimal_model.joblib')
joblib.dump(gmm_optimal, model_filepath)
print(f"\n[OK] Model saved to: {model_filepath}")

In [ ]:
# =============================================================================
# PHASE 8: CLUSTER ANALYSIS AND INTERPRETATION
# =============================================================================

print("=" * 70)
print("CLUSTER ANALYSIS AND INTERPRETATION")
print("=" * 70)

# Get cluster labels for full dataset
full_labels = gmm_optimal.predict(X_scaled)
data['cluster'] = full_labels

cluster_counts = pd.Series(full_labels).value_counts().sort_index()
cluster_proportions = cluster_counts / len(full_labels) * 100

print(f"\n[INFO] CLUSTER DISTRIBUTION:")
print("-" * 40)
for cluster, count in cluster_counts.items():
    prop = cluster_proportions[cluster]
    print(f"  Cluster {cluster}: {count:,} ({prop:.1f}%)")

# Calculate cluster profiles
cluster_profiles = data.groupby('cluster')[FEATURE_COLUMNS].mean()

print("\n[INFO] CLUSTER PROFILES (Mean Values):")
print("-" * 100)
display(cluster_profiles.round(2))

# Visualize cluster profiles as heatmap
fig, ax = plt.subplots(figsize=(14, 8))

# Normalize for better visualization
cluster_profiles_normalized = (cluster_profiles - cluster_profiles.min()) / (cluster_profiles.max() - cluster_profiles.min())

sns.heatmap(cluster_profiles_normalized.T, 
            annot=cluster_profiles.T.round(1), 
            fmt='', 
            cmap='RdYlGn',
            center=0.5,
            linewidths=0.5,
            cbar_kws={'label': 'Normalized Value'},
            ax=ax)

ax.set_xlabel('Cluster', fontsize=12)
ax.set_ylabel('Health Indicator', fontsize=12)
ax.set_title('GMM Cluster Profiles: Mean Values by Health Indicator\n(NHANES Data)', fontweight='bold')

plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, 'plots', '04_cluster_profiles_heatmap.png'), 
            dpi=150, bbox_inches='tight')
plt.show()
print("\n[OK] Figure saved: figures/plots/04_cluster_profiles_heatmap.png")

# Save cluster profiles
cluster_profiles.to_csv(os.path.join(OUTPUT_DIR, 'cluster_profiles', 'gmm_cluster_profiles.csv'))
print("[OK] Cluster profiles saved to: output_v2/cluster_profiles/")

In [ ]:
# =============================================================================
# PHASE 9: CLUSTER VISUALIZATION IN REDUCED DIMENSIONS
# =============================================================================

print("=" * 70)
print("CLUSTER VISUALIZATION")
print("=" * 70)

# Visualize clusters in PCA and t-SNE space
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# PCA visualization
scatter1 = axes[0].scatter(X_pca[:, 0], X_pca[:, 1], 
                            c=data['cluster'], cmap='viridis',
                            alpha=0.6, s=30, edgecolor='white', linewidth=0.3)
axes[0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
axes[0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
axes[0].set_title('GMM Clusters in PCA Space (NHANES)', fontweight='bold')
plt.colorbar(scatter1, ax=axes[0], label='Cluster')

# Add cluster centroids
for cluster in range(best_params['n_components']):
    mask = data['cluster'] == cluster
    centroid_pca = X_pca[mask].mean(axis=0)
    axes[0].scatter(centroid_pca[0], centroid_pca[1], c='red', s=200, marker='X', 
                    edgecolor='white', linewidth=2, zorder=10)

# t-SNE visualization
scatter2 = axes[1].scatter(X_tsne[:, 0], X_tsne[:, 1], 
                            c=data['cluster'], cmap='viridis',
                            alpha=0.6, s=30, edgecolor='white', linewidth=0.3)
axes[1].set_xlabel('t-SNE Dimension 1')
axes[1].set_ylabel('t-SNE Dimension 2')
axes[1].set_title('GMM Clusters in t-SNE Space (NHANES)', fontweight='bold')
plt.colorbar(scatter2, ax=axes[1], label='Cluster')

# Add cluster centroids
for cluster in range(best_params['n_components']):
    mask = data['cluster'] == cluster
    centroid_tsne = X_tsne[mask].mean(axis=0)
    axes[1].scatter(centroid_tsne[0], centroid_tsne[1], c='red', s=200, marker='X', 
                    edgecolor='white', linewidth=2, zorder=10)

plt.suptitle('GMM Clustering Results on NHANES Health Data', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, 'plots', '05_gmm_clustering_results.png'), 
            dpi=150, bbox_inches='tight')
plt.show()
print("\n[OK] Figure saved: figures/plots/05_gmm_clustering_results.png")

## Phase 10: Model Evaluation

This section evaluates the GMM model performance using multiple clustering metrics:

- **Silhouette Score**: Measures cluster cohesion and separation (-1 to 1, higher is better)

- **Calinski-Harabasz Index**: Ratio of between-cluster to within-cluster variance (higher is better)

- **Davies-Bouldin Index**: Average similarity between clusters (lower is better)

- **BIC/AIC**: Model selection criteria balancing fit and complexity (lower is better)


In [ ]:
# =============================================================================
# PHASE 10: MODEL EVALUATION
# =============================================================================

print("=" * 70)
print("MODEL EVALUATION")
print("=" * 70)

# Evaluate on training and test sets
train_labels = gmm_optimal.predict(X_train)
test_labels = gmm_optimal.predict(X_test)

def evaluate_gmm(X, labels, model):
    """Comprehensive evaluation of GMM model performance."""
    metrics = {}
    metrics['silhouette'] = silhouette_score(X, labels)
    metrics['calinski_harabasz'] = calinski_harabasz_score(X, labels)
    metrics['davies_bouldin'] = davies_bouldin_score(X, labels)
    metrics['bic'] = model.bic(X)
    metrics['aic'] = model.aic(X)
    metrics['log_likelihood'] = model.score(X)
    return metrics

train_eval = evaluate_gmm(X_train, train_labels, gmm_optimal)
test_eval = evaluate_gmm(X_test, test_labels, gmm_optimal)

print(f"\n[INFO] MODEL EVALUATION METRICS:")
print("-" * 60)
print(f"{'Metric':<25} {'Training':>12} {'Test':>12}")
print("-" * 60)
for key in train_eval:
    print(f"{key:<25} {train_eval[key]:>12.4f} {test_eval[key]:>12.4f}")

# Final evaluation on full dataset
full_eval = evaluate_gmm(X_scaled, full_labels, gmm_optimal)

print(f"\n[INFO] FINAL MODEL PERFORMANCE (Full Dataset):")
print("-" * 60)
print(f"  Silhouette Score: {full_eval['silhouette']:.4f}")
print(f"  Calinski-Harabasz Index: {full_eval['calinski_harabasz']:.2f}")
print(f"  Davies-Bouldin Index: {full_eval['davies_bouldin']:.4f}")
print(f"  BIC Score: {full_eval['bic']:.2f}")
print(f"  AIC Score: {full_eval['aic']:.2f}")

In [ ]:
# =============================================================================
# PHASE 11: PROBABILISTIC MEMBERSHIP ANALYSIS
# =============================================================================

print("=" * 70)
print("PROBABILISTIC MEMBERSHIP ANALYSIS")
print("=" * 70)

# Get membership probabilities
membership_probs = gmm_optimal.predict_proba(X_scaled)

print(f"\n[INFO] CLUSTER MEMBERSHIP PROBABILITIES:")
print("-" * 60)

for i in range(best_params['n_components']):
    probs = membership_probs[:, i]
    high_conf = (probs >= 0.8).sum()
    print(f"\n  Cluster {i}:")
    print(f"    Mean Probability:   {probs.mean():.4f}")
    print(f"    Std Deviation:      {probs.std():.4f}")
    print(f"    High Confidence (>=0.8): {high_conf:,} ({100*high_conf/len(probs):.1f}%)")

# Certainty analysis
max_probs = membership_probs.max(axis=1)
high_conf_total = (max_probs >= 0.8).sum()
mod_conf = ((max_probs >= 0.5) & (max_probs < 0.8)).sum()
low_conf = (max_probs < 0.5).sum()

print(f"\n[INFO] CLUSTER ASSIGNMENT CERTAINTY:")
print(f"  Very High Confidence (>=0.8): {high_conf_total:,} ({100*high_conf_total/len(max_probs):.1f}%)")
print(f"  Moderate Confidence (0.5-0.8): {mod_conf:,} ({100*mod_conf/len(max_probs):.1f}%)")
print(f"  Low Confidence (<0.5): {low_conf:,} ({100*low_conf/len(max_probs):.1f}%)")

# Add probabilities to dataframe
for i in range(best_params['n_components']):
    data[f'prob_cluster_{i}'] = membership_probs[:, i]

# Save predictions
predictions_df = data[['respondent_id', 'cluster', 'bmi', 'systolic_bp_mmHg', 
                       'fasting_glucose_mg_dL', 'phq9_total_score'] + 
                      [f'prob_cluster_{i}' for i in range(best_params['n_components'])]]
predictions_df.to_csv(os.path.join(OUTPUT_DIR, 'predictions', 'gmm_cluster_predictions.csv'), index=False)
print(f"\n[OK] Predictions saved to: output_v2/predictions/")

## Phase 12: Conclusions and Future Work

This final section provides a comprehensive summary of the project including:

- **Project Methodology and Key Findings**: Summary of the GMM approach, optimal cluster selection, and main analytical results.

- **Cluster Characteristics and Health Phenotype Descriptions**: Detailed profiles of each identified subpopulation with their distinctive health characteristics.

- **Public Health Implications**: Discussion of how discovered phenotypes can inform targeted interventions and health policy decisions.

- **Limitations and Future Research Directions**: Honest assessment of study limitations and potential extensions for future investigations.

- **Reproducibility and Configuration**: Project metrics, parameters, and configuration settings saved for complete reproducibility.


In [ ]:
# =============================================================================
# PHASE 12: CONCLUSIONS AND FUTURE WORK
# =============================================================================

print("=" * 70)
print("CONCLUSIONS AND SUMMARY")
print("=" * 70)

n_clusters = best_params['n_components']
silhouette_final = full_eval['silhouette']
bic_final = full_eval['bic']

print(f"""
PROJECT SUMMARY
---------------
This project applied Gaussian Mixture Models (GMM) to identify latent 
subpopulations in NHANES health data (5,000 respondents, 47 variables).

METHODOLOGY:
- Dataset: NHANES (National Health and Nutrition Examination Survey)
- Algorithm: Gaussian Mixture Models (GMM)
- Hyperparameter Tuning: Grid search with BIC optimization
- Best Configuration:
  * Number of clusters: {n_clusters}
  * Covariance type: {best_params['covariance_type']}
  * Regularization: {best_params['reg_covar']}

KEY FINDINGS:
1. Optimal Number of Clusters: {n_clusters}
   - BIC Score: {bic_final:.2f}
   - Silhouette Score: {silhouette_final:.4f}

2. Cluster Characteristics:
   - Identified {n_clusters} distinct health phenotypes
   - {100*high_conf_total/len(max_probs):.1f}% high-confidence assignments
   - Clear separation between risk profiles

CLUSTER PROFILES:
-----------------""")

for cluster in range(n_clusters):
    cluster_data = data[data['cluster'] == cluster]
    print(f"\n  Cluster {cluster} ({len(cluster_data):,} individuals, {100*len(cluster_data)/len(data):.1f}%):")
    print(f"    Mean BMI: {cluster_data['bmi'].mean():.1f}")
    print(f"    Mean Systolic BP: {cluster_data['systolic_bp_mmHg'].mean():.1f} mmHg")
    print(f"    Mean Glucose: {cluster_data['fasting_glucose_mg_dL'].mean():.1f} mg/dL")
    print(f"    Mean PHQ-9 Score: {cluster_data['phq9_total_score'].mean():.1f}")

print(f"""
PUBLIC HEALTH IMPLICATIONS:
- The {n_clusters} clusters represent distinct health phenotypes with different
  risk profiles and intervention needs
- Probabilistic assignments allow for uncertainty-aware decision making
- This approach can support targeted intervention design and resource allocation

LIMITATIONS:
- Cross-sectional data limits causal inference
- External validation with independent datasets recommended
- Clinical validation required before operational deployment

FUTURE WORK:
- Compare GMM with other clustering methods (hierarchical, DBSCAN)
- Longitudinal analysis using NHANES temporal data
- Semi-supervised GMM with known health outcomes
- Integration with clinical risk scores
""")

# Save final configuration
config = {
    'student_id': 'SDS6/46982/2025',
    'course': 'SDS6217 Advanced Machine Learning',
    'institution': 'University of Nairobi',
    'dataset': 'NHANES Health Data',
    'n_samples': int(data.shape[0]),
    'n_features': len(FEATURE_COLUMNS),
    'best_params': best_params,
    'training_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'metrics': {
        'bic': float(bic_final),
        'aic': float(full_eval['aic']),
        'silhouette': float(silhouette_final),
        'calinski_harabasz': float(full_eval['calinski_harabasz']),
        'davies_bouldin': float(full_eval['davies_bouldin'])
    },
'n_clusters': n_clusters
}

with open(os.path.join(OUTPUT_DIR, 'metrics', 'project_config.json'), 'w') as f:
    json.dump(config, f, indent=2)

print("[OK] Configuration saved to: output_v2/metrics/")

print("\n" + "=" * 70)
print("PROJECT COMPLETE")
print("=" * 70)
print(f"Student ID: SDS6/46982/2025")
print(f"Course: SDS6217 Advanced Machine Learning")
print(f"Institution: University of Nairobi")
print(f"Dataset: NHANES Health Data (5,000 samples, 47 variables)")
print(f"Clusters Found: {n_clusters}")
print("=" * 70)

## References

1. National Center for Health Statistics. (2017-2018). National Health and Nutrition Examination Survey. Centers for Disease Control and Prevention. https://www.cdc.gov/nchs/nhanes/

2. McLachlan, G.J., & Peel, D. (2000). Finite Mixture Models. John Wiley & Sons.

3. Bishop, C.M. (2006). Pattern Recognition and Machine Learning. Springer.

4. Schwarz, G. (1978). Estimating the dimension of a model. Annals of Statistics, 6(2), 461-464.

5. Akaike, H. (1974). A new look at the statistical model identification. IEEE Transactions on Automatic Control, 19(6), 716-723.

---  

**Author:** Cavin Otieno  
**Student ID:** SDS6/46982/2025  
**MSc Public Health Data Science - SDS6217 Advanced Machine Learning**  
**University of Nairobi**

## Phase 13: BIC/AIC Model Selection Analysis

This section provides comprehensive analysis of model selection criteria:

- **BIC (Bayesian Information Criterion)**: Penalizes model complexity while rewarding goodness of fit. Lower BIC indicates a better model.

- **AIC (Akaike Information Criterion)**: Based on information theory, AIC estimates the relative quality of models by balancing fit against complexity.

- **Elbow Method Visualization**: Plots BIC and AIC across different numbers of components.

- **Covariance Type Comparison**: Compares model performance across different covariance structures.

- **Comprehensive Metrics**: Evaluates using 5 cluster quality metrics (BIC, AIC, Silhouette, Calinski-Harabasz, Davies-Bouldin).


In [ ]:
# =============================================================================
# PHASE 13: MODEL SELECTION - BIC/AIC CURVES
# =============================================================================
print("=" * 70)
print("MODEL SELECTION: COMPREHENSIVE GMM EVALUATION")
print("=" * 70)
# Import additional metrics
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
# Define cluster range to evaluate
n_components_range = range(2, 16)
# Storage for metrics
bic_scores = []
aic_scores = []
silhouette_scores = []
calinski_scores = []
davies_bouldin_scores = []
print(f"\n[INFO] Evaluating models with k=2 to k=15 clusters...")
print("-" * 70)
print(f"{'k':^4} | {'BIC':^12} | {'AIC':^12} | {'Silhouette':^12} | {'Calinski':^12} | {'Davies':^10}")
print("-" * 70)
for n_components in n_components_range:
    # Create and fit GMM
    gmm_temp = GaussianMixture(
        n_components=n_components,
        covariance_type='full',
        n_init=5,
        random_state=42,
        max_iter=200,
        reg_covar=1e-6
    )
    gmm_temp.fit(X_scaled)
    
    # Get predictions
    labels_temp = gmm_temp.predict(X_scaled)
    
    # Calculate all metrics
    bic = gmm_temp.bic(X_scaled)
    aic = gmm_temp.aic(X_scaled)
    silhouette = silhouette_score(X_scaled, labels_temp)
    calinski = calinski_harabasz_score(X_scaled, labels_temp)
    davies = davies_bouldin_score(X_scaled, labels_temp)
    
    # Store scores
    bic_scores.append(bic)
    aic_scores.append(aic)
    silhouette_scores.append(silhouette)
    calinski_scores.append(calinski)
    davies_bouldin_scores.append(davies)
    
    # Print progress
    print(f"{n_components:^4} | {bic:^12.2f} | {aic:^12.2f} | {silhouette:^12.4f} | {calinski:^12.2f} | {davies:^10.4f}")
print("-" * 70)
# Find optimal number of components for each metric
best_bic_n = list(n_components_range)[np.argmin(bic_scores)]
best_aic_n = list(n_components_range)[np.argmin(aic_scores)]
best_silhouette_n = list(n_components_range)[np.argmax(silhouette_scores)]
best_calinski_n = list(n_components_range)[np.argmax(calinski_scores)]
best_davies_n = list(n_components_range)[np.argmin(davies_bouldin_scores)]
print(f"\n[INFO] OPTIMAL COMPONENTS BY METRIC:")
print(f"  • BIC (lower is better):          k = {best_bic_n}")
print(f"  • AIC (lower is better):          k = {best_aic_n}")
print(f"  • Silhouette (higher is better):  k = {best_silhouette_n}")
print(f"  • Calinski-Harabasz (higher is better): k = {best_calinski_n}")
print(f"  • Davies-Bouldin (lower is better):    k = {best_davies_n}")
# Use best_params from previous analysis
optimal_k = best_params['n_components']
print(f"\n[INFO] SELECTED OPTIMAL K: {optimal_k} (from best_params)")
# Create comprehensive visualization
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('GMM Model Selection Analysis\nEvaluating Clusters k=2 to k=15', 
             fontsize=16, fontweight='bold', y=1.02)
# Plot 1: BIC and AIC curves
ax1 = axes[0, 0]
ax1.plot(list(n_components_range), bic_scores, 'b-o', linewidth=2, markersize=8, label='BIC')
ax1.plot(list(n_components_range), aic_scores, 'r-s', linewidth=2, markersize=8, label='AIC')
ax1.axvline(x=optimal_k, color='green', linestyle='--', linewidth=2, label=f'Selected k={optimal_k}')
ax1.axvline(x=best_bic_n, color='blue', linestyle=':', alpha=0.7, label=f'Best BIC={best_bic_n}')
ax1.axvline(x=best_aic_n, color='red', linestyle=':', alpha=0.7, label=f'Best AIC={best_aic_n}')
ax1.set_xlabel('Number of Clusters (k)', fontsize=12)
ax1.set_ylabel('Information Criterion Score', fontsize=12)
ax1.set_title('BIC and AIC\n(Lower is Better)', fontsize=14, fontweight='bold')
ax1.legend(fontsize=9, loc='best')
ax1.grid(True, alpha=0.3)
ax1.set_xticks(list(n_components_range))
# Plot 2: Silhouette Score
ax2 = axes[0, 1]
ax2.plot(list(n_components_range), silhouette_scores, 'g-^', linewidth=2, markersize=8, color='darkgreen')
ax2.axvline(x=optimal_k, color='red', linestyle='--', linewidth=2, label=f'Selected k={optimal_k}')
ax2.axvline(x=best_silhouette_n, color='green', linestyle=':', alpha=0.7, label=f'Best k={best_silhouette_n}')
ax2.fill_between(list(n_components_range), silhouette_scores, alpha=0.2, color='green')
ax2.set_xlabel('Number of Clusters (k)', fontsize=12)
ax2.set_ylabel('Silhouette Score', fontsize=12)
ax2.set_title('Silhouette Score\n(Higher is Better)', fontsize=14, fontweight='bold')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)
ax2.set_xticks(list(n_components_range))
# Plot 3: Calinski-Harabasz Index
ax3 = axes[0, 2]
ax3.plot(list(n_components_range), calinski_scores, 'm-v', linewidth=2, markersize=8, color='purple')
ax3.axvline(x=optimal_k, color='red', linestyle='--', linewidth=2, label=f'Selected k={optimal_k}')
ax3.axvline(x=best_calinski_n, color='purple', linestyle=':', alpha=0.7, label=f'Best k={best_calinski_n}')
ax3.fill_between(list(n_components_range), calinski_scores, alpha=0.2, color='purple')
ax3.set_xlabel('Number of Clusters (k)', fontsize=12)
ax3.set_ylabel('Calinski-Harabasz Index', fontsize=12)
ax3.set_title('Calinski-Harabasz Index\n(Higher is Better)', fontsize=14, fontweight='bold')
ax3.legend(fontsize=10)
ax3.grid(True, alpha=0.3)
ax3.set_xticks(list(n_components_range))
# Plot 4: Davies-Bouldin Index
ax4 = axes[1, 0]
ax4.plot(list(n_components_range), davies_bouldin_scores, 'c-d', linewidth=2, markersize=8, color='teal')
ax4.axvline(x=optimal_k, color='red', linestyle='--', linewidth=2, label=f'Selected k={optimal_k}')
ax4.axvline(x=best_davies_n, color='teal', linestyle=':', alpha=0.7, label=f'Best k={best_davies_n}')
ax4.fill_between(list(n_components_range), davies_bouldin_scores, alpha=0.2, color='teal')
ax4.set_xlabel('Number of Clusters (k)', fontsize=12)
ax4.set_ylabel('Davies-Bouldin Index', fontsize=12)
ax4.set_title('Davies-Bouldin Index\n(Lower is Better)', fontsize=14, fontweight='bold')
ax4.legend(fontsize=10)
ax4.grid(True, alpha=0.3)
ax4.set_xticks(list(n_components_range))
# Plot 5: Combined normalized metrics
ax5 = axes[1, 1]
# Normalize all metrics to [0, 1] for comparison
bic_norm = (np.array(bic_scores) - min(bic_scores)) / (max(bic_scores) - min(bic_scores))
aic_norm = (np.array(aic_scores) - min(aic_scores)) / (max(aic_scores) - min(aic_scores))
sil_norm = (np.array(silhouette_scores) - min(silhouette_scores)) / (max(silhouette_scores) - min(silhouette_scores))
calinski_norm = (np.array(calinski_scores) - min(calinski_scores)) / (max(calinski_scores) - min(calinski_scores))
davies_norm = (np.array(davies_bouldin_scores) - min(davies_bouldin_scores)) / (max(davies_bouldin_scores) - min(davies_bouldin_scores))
# For lower-is-better metrics, invert so higher is better
bic_norm_inv = 1 - bic_norm
aic_norm_inv = 1 - aic_norm
davies_norm_inv = 1 - davies_norm
# Calculate composite score
composite = (bic_norm_inv + aic_norm_inv + sil_norm + calinski_norm + davies_norm_inv) / 5
ax5.plot(list(n_components_range), bic_norm_inv, 'b--', linewidth=1.5, alpha=0.6, label='BIC (inv)')
ax5.plot(list(n_components_range), aic_norm_inv, 'r--', linewidth=1.5, alpha=0.6, label='AIC (inv)')
ax5.plot(list(n_components_range), sil_norm, 'g-', linewidth=2, label='Silhouette')
ax5.plot(list(n_components_range), calinski_norm, 'm-', linewidth=2, label='Calinski')
ax5.plot(list(n_components_range), davies_norm_inv, 'c--', linewidth=1.5, alpha=0.6, label='Davies (inv)')
ax5.plot(list(n_components_range), composite, 'k-', linewidth=3, label='Composite Score')
ax5.axvline(x=optimal_k, color='red', linestyle='--', linewidth=2, label=f'Selected k={optimal_k}')
ax5.set_xlabel('Number of Clusters (k)', fontsize=12)
ax5.set_ylabel('Normalized Score (0-1)', fontsize=12)
ax5.set_title('Normalized Metrics Comparison\n(Higher is Better for All)', fontsize=14, fontweight='bold')
ax5.legend(fontsize=8, loc='best', ncol=2)
ax5.grid(True, alpha=0.3)
ax5.set_xticks(list(n_components_range))
# Plot 6: Summary table
ax6 = axes[1, 2]
ax6.axis('off')
summary_text = f"""
MODEL SELECTION SUMMARY
{'='*40}
Dataset: X_scaled ({X_scaled.shape[0]:,} samples, {X_scaled.shape[1]} features)
Cluster Range Tested: k = 2 to 15
OPTIMAL K BY METRIC:
  • BIC:          k = {best_bic_n}
  • AIC:          k = {best_aic_n}
  • Silhouette:   k = {best_silhouette_n}
  • Calinski:     k = {best_calinski_n}
  • Davies:       k = {best_davies_n}
SELECTED OPTIMAL K: {optimal_k}
METRICS AT k={optimal_k}:
  • BIC Score:          {bic_scores[optimal_k-2]:.2f}
  • AIC Score:          {aic_scores[optimal_k-2]:.2f}
  • Silhouette Score:   {silhouette_scores[optimal_k-2]:.4f}
  • Calinski-Harabasz:  {calinski_scores[optimal_k-2]:.2f}
  • Davies-Bouldin:     {davies_bouldin_scores[optimal_k-2]:.4f}
"""
ax6.text(0.1, 0.95, summary_text, transform=ax6.transAxes, fontsize=11,
         verticalalignment='top', fontfamily='monospace',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
plt.tight_layout()
# Ensure output directory exists
import os
os.makedirs('output_v2/figures/plots/', exist_ok=True)
# Save figure
plt.savefig('output_v2/figures/plots/06_bic_aic_analysis.png', dpi=300, bbox_inches='tight')
plt.show()
print(f"\n[OK] Figure saved: output_v2/figures/plots/06_bic_aic_analysis.png")
# Save detailed results to CSV
model_selection_df = pd.DataFrame({
    'n_components': list(n_components_range),
    'bic_score': bic_scores,
    'aic_score': aic_scores,
    'silhouette_score': silhouette_scores,
    'calinski_harabasz_index': calinski_scores,
    'davies_bouldin_index': davies_bouldin_scores,
    'optimal_by_bic': [n == best_bic_n for n in n_components_range],
    'optimal_by_aic': [n == best_aic_n for n in n_components_range],
    'optimal_by_silhouette': [n == best_silhouette_n for n in n_components_range],
    'optimal_by_calinski': [n == best_calinski_n for n in n_components_range],
    'optimal_by_davies': [n == best_davies_n for n in n_components_range]
})
os.makedirs('output_v2/metrics/', exist_ok=True)
model_selection_df.to_csv('output_v2/metrics/model_selection_results.csv', index=False)
print(f"[OK] Results saved: output_v2/metrics/model_selection_results.csv")
# Print final summary
print("\n" + "=" * 70)
print("MODEL SELECTION COMPLETE")
print("=" * 70)
print(f"\nSelected optimal number of clusters: k = {optimal_k}")
print(f"\nMetrics saved to: output_v2/metrics/model_selection_results.csv")
print(f"Visualization saved to: output_v2/figures/plots/06_bic_aic_analysis.png")
print("=" * 70)


## Phase 14: Radar Charts for Cluster Profiles

This section creates radar charts to visualize cluster profiles across multiple health dimensions.


In [ ]:
# =============================================================================
# PHASE 14: RADAR CHARTS FOR CLUSTER PROFILES
# =============================================================================
print("=" * 70)
print("RADAR CHARTS FOR CLUSTER PROFILES")
print("=" * 70)
import matplotlib.pyplot as plt
import numpy as np
def create_radar_chart(ax, categories, values, title, color, alpha=0.25):
    """Create a radar chart for cluster profile visualization."""
    N = len(categories)
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # Complete the loop
    values = list(values)  # Convert to list if numpy array
    values = values + [values[0]]  # Complete the loop
    ax.plot(angles, values, linewidth=2, linestyle='solid', color=color)
    ax.fill(angles, values, color=color, alpha=alpha)
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(categories, fontsize=9)
    ax.set_title(title, fontsize=12, fontweight='bold', pad=20)
# Define features for radar chart
radar_features = ['age', 'bmi', 'systolic_bp_mmHg', 'fasting_glucose_mg_dL', 'phq9_total_score']
feature_labels = ['Age', 'BMI', 'Systolic BP', 'Glucose', 'PHQ-9']
# Normalize features
normalized_data = data.copy()
for col in radar_features:
    min_val = data[col].min()
    max_val = data[col].max()
    if max_val > min_val:
        normalized_data[col] = (data[col] - min_val) / (max_val - min_val)
    else:
        normalized_data[col] = 0.5
# Calculate cluster profiles
cluster_profiles = normalized_data.groupby(data['cluster'])[radar_features].mean()
# Create radar charts
n_clusters = len(cluster_profiles)
colors = plt.cm.Set2(np.linspace(0, 1, n_clusters))
fig, axes = plt.subplots(1, n_clusters, figsize=(5*n_clusters, 5), subplot_kw=dict(polar=True))
if n_clusters == 1:
    axes = [axes]
for idx, (cluster_id, profile) in enumerate(cluster_profiles.iterrows()):
    cluster_count = len(data[data['cluster'] == cluster_id])
    create_radar_chart(
        axes[idx],
        feature_labels,
        profile.values,
        f'Cluster {cluster_id}\n(n={cluster_count})',
        colors[idx]
    )
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, 'plots', '07_radar_charts.png'), dpi=300, bbox_inches='tight')
plt.show()
print(f"\n[OK] Figure saved: {os.path.join(FIGURES_DIR, 'plots', '07_radar_charts.png')}")


### Combined Cluster Comparison Radar Chart

This section provides a combined radar chart comparing all cluster profiles on a single visualization for direct comparison.


In [ ]:
# =============================================================================
# PHASE 14 (continued): COMBINED RADAR CHART - CLUSTER PROFILE COMPARISON
# =============================================================================
print("=" * 70)
print("RADAR CHART: CLUSTER PROFILE COMPARISON")
print("=" * 70)
# Select key health indicators for radar chart
radar_features = ['bmi', 'systolic_bp_mmHg', 'fasting_glucose_mg_dL', 'phq9_total_score', 'age']
radar_features = [f for f in radar_features if f in data.columns]
# Normalize cluster means to 0-1 scale for comparison
cluster_means = data.groupby('cluster')[radar_features].mean()
feature_mins = data[radar_features].min()
feature_maxs = data[radar_features].max()
cluster_means_normalized = (cluster_means - feature_mins) / (feature_maxs - feature_mins)
# Create radar chart
fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
# Number of features
num_vars = len(radar_features)
angles = np.linspace(0, 2 * np.pi, num_vars, endpoint=False).tolist()
angles += angles[:1]  # Complete the loop
# Colors for each cluster
colors = plt.cm.viridis(np.linspace(0, 1, best_params['n_components']))
# Plot each cluster
for i in range(best_params['n_components']):
    values = cluster_means_normalized.loc[i].tolist()
    values += values[:1]  # Complete the loop
    ax.plot(angles, values, 'o-', linewidth=2, label=f'Cluster {i} (n={cluster_sizes[i]:,})', color=colors[i])
    ax.fill(angles, values, alpha=0.15, color=colors[i])
# Set the labels
ax.set_xticks(angles[:-1])
labels = ['BMI', 'Systolic BP', 'Glucose', 'PHQ-9 Score', 'Age']
labels = labels[:len(radar_features)]
ax.set_xticklabels(labels, fontsize=12)
ax.set_ylim(0, 1)
ax.set_title('Cluster Profiles: Normalized Health Indicators\n', fontsize=14, fontweight='bold')
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=10)
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, 'plots', '08_combined_radar.png'), dpi=150, bbox_inches='tight')
plt.show()
print(f"\n[OK] Figure saved: {os.path.join(FIGURES_DIR, 'plots', '08_combined_radar.png')}")
print("=" * 70)


### Cluster Size Distribution and Quality Metrics

This section analyzes cluster sizes and calculates quality metrics including compactness and separation.


In [ ]:
# =============================================================================
# PHASE 14 (continued): CLUSTER SIZE DISTRIBUTION AND QUALITY METRICS
# =============================================================================
print("=" * 70)
print("CLUSTER SIZE DISTRIBUTION AND QUALITY METRICS")
print("=" * 70)
# Calculate cluster sizes
cluster_sizes = data['cluster'].value_counts().sort_index()
cluster_proportions = (cluster_sizes / len(data) * 100).round(2)
print("\n[INFO] Cluster Size Distribution:")
for cluster in range(best_params['n_components']):
    size = cluster_sizes[cluster]
    proportion = cluster_proportions[cluster]
    print(f"  Cluster {cluster}: {size:,} samples ({proportion}%)")
# Calculate cluster quality metrics
cluster_metrics = {}
for cluster in range(best_params['n_components']):
    mask = data['cluster'] == cluster
    cluster_data = X_scaled[mask]
    
    # Calculate compactness (average distance to centroid)
    centroid = cluster_data.mean(axis=0)
    distances = np.sqrt(((cluster_data - centroid) ** 2).sum(axis=1))
    compactness = distances.mean()
    
    # Calculate separation (distance to nearest other cluster centroid)
    min_separation = float('inf')
    for other_cluster in range(best_params['n_components']):
        if other_cluster != cluster:
            other_mask = data['cluster'] == other_cluster
            other_centroid = X_scaled[other_mask].mean(axis=0)
            sep = np.sqrt(((centroid - other_centroid) ** 2).sum())
            min_separation = min(min_separation, sep)
    
    cluster_metrics[cluster] = {
        'size': int(size),
        'proportion': float(proportion),
        'compactness': float(compactness),
        'separation': float(min_separation),
        'compactness_separation_ratio': float(compactness / min_separation) if min_separation > 0 else 0
    }
# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))
# Cluster size bar chart
colors = plt.cm.viridis(np.linspace(0, 1, best_params['n_components']))
bars = axes[0].bar(range(best_params['n_components']), cluster_sizes.values, color=colors, edgecolor='black')
axes[0].set_xlabel('Cluster', fontsize=12)
axes[0].set_ylabel('Number of Samples', fontsize=12)
axes[0].set_title('Cluster Size Distribution', fontweight='bold')
axes[0].set_xticks(range(best_params['n_components']))
# Add count labels on bars
for bar, size in zip(bars, cluster_sizes.values):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 50, 
                 f'{size:,}', ha='center', va='bottom', fontsize=10, fontweight='bold')
# Compactness vs Separation
compactness_vals = [cluster_metrics[c]['compactness'] for c in range(best_params['n_components'])]
separation_vals = [cluster_metrics[c]['separation'] for c in range(best_params['n_components'])]
x_pos = np.arange(best_params['n_components'])
width = 0.35
bars1 = axes[1].bar(x_pos - width/2, compactness_vals, width, label='Compactness (lower=better)', color='coral', edgecolor='black')
bars2 = axes[1].bar(x_pos + width/2, separation_vals, width, label='Separation (higher=better)', color='steelblue', edgecolor='black')
axes[1].set_xlabel('Cluster', fontsize=12)
axes[1].set_ylabel('Score', fontsize=12)
axes[1].set_title('Cluster Quality Metrics: Compactness vs Separation', fontweight='bold')
axes[1].set_xticks(x_pos)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig(os.path.join(FIGURES_DIR, 'plots', '09_cluster_quality_metrics.png'), dpi=150, bbox_inches='tight')
plt.show()
# Save cluster metrics
metrics_df = pd.DataFrame(cluster_metrics).T
metrics_df.index.name = 'cluster'
os.makedirs('output_v2/metrics/', exist_ok=True)
metrics_df.to_csv(os.path.join(OUTPUT_DIR, 'metrics', 'cluster_quality_metrics.csv'))
print(f"\n[OK] Figure saved: {os.path.join(FIGURES_DIR, 'plots', '09_cluster_quality_metrics.png')}")
print(f"[OK] Metrics saved: output_v2/metrics/cluster_quality_metrics.csv")
print("=" * 70)


## Phase 15: Feature Distribution by Cluster

This section visualizes the distribution of key health features within each cluster.


In [ ]:
# PHASE 15: FEATURE DISTRIBUTION BY CLUSTERprint("=" * 70)print("FEATURE DISTRIBUTION BY CLUSTER")print("=" * 70)import seaborn as sns# Features to plotplot_features = ['age', 'bmi', 'systolic_bp_mmHg', 'fasting_glucose_mg_dL', 'phq9_total_score']plot_titles = ['Age (years)', 'BMI (kg/m²)', 'Systolic BP (mmHg)', 'Fasting Glucose (mg/dL)', 'PHQ-9 Score']# Create box plotsfig, axes = plt.subplots(2, 3, figsize=(15, 10))axes = axes.flatten()for idx, (feature, title) in enumerate(zip(plot_features, plot_titles)):    sns.boxplot(data=data, x='cluster', y=feature, ax=axes[idx], palette='Set2')    axes[idx].set_title(f'{title} by Cluster', fontsize=12, fontweight='bold')axes[5].axis('off')plt.tight_layout()plt.savefig(os.path.join(OUTPUT_DIR, 'figures', 'plots', '09_feature_boxplots.png'), dpi=300, bbox_inches='tight')plt.show()print(f"[OK] Figure saved: {os.path.join(OUTPUT_DIR, 'figures', 'plots', '09_feature_boxplots.png')}")# Create violin plotsfig, axes = plt.subplots(2, 3, figsize=(15, 10))axes = axes.flatten()for idx, (feature, title) in enumerate(zip(plot_features, plot_titles)):    sns.violinplot(data=data, x='cluster', y=feature, ax=axes[idx], palette='Set2', inner='box')    axes[idx].set_title(f'{title} Distribution by Cluster', fontsize=12, fontweight='bold')axes[5].axis('off')plt.tight_layout()plt.savefig(os.path.join(OUTPUT_DIR, 'figures', 'plots', '10_feature_violin.png'), dpi=300, bbox_inches='tight')plt.show()print(f"[OK] Figure saved: {os.path.join(OUTPUT_DIR, 'figures', 'plots', '10_feature_violin.png')}")

## Phase 16: Cluster Size and Proportion Analysis

This section analyzes the distribution of samples across clusters.


In [ ]:
# PHASE 16: CLUSTER SIZE AND PROPORTION ANALYSISprint("=" * 70)print("CLUSTER SIZE AND PROPORTION ANALYSIS")print("=" * 70)# Calculate cluster sizes and proportionscluster_sizes = data['cluster'].value_counts().sort_index()cluster_proportions = (cluster_sizes / len(data)) * 100print("\nCluster Distribution:")print("-" * 50)print(f"{'Cluster':<10} {'Count':<10} {'Proportion':<15}")print("-" * 50)for cluster in cluster_sizes.index:    print(f"{cluster:<10} {cluster_sizes[cluster]:<10} {cluster_proportions[cluster]:.2f}%")print("-" * 50)print(f"{'Total':<10} {len(data):<10} {100.0:.2f}%")# Create visualizationsfig, axes = plt.subplots(1, 2, figsize=(14, 6))# Pie chartcolors = plt.cm.Set2(np.linspace(0, 1, len(cluster_sizes)))axes[0].pie(cluster_sizes, labels=[f'Cluster {i}\n(n={v})' for i, v in cluster_sizes.items()],            autopct='%1.1f%%', colors=colors, explode=[0.02]*len(cluster_sizes),            shadow=True, startangle=90)axes[0].set_title('Cluster Distribution (Pie Chart)', fontsize=14, fontweight='bold')# Bar chartbars = axes[1].bar(cluster_sizes.index, cluster_sizes.values, color=colors, edgecolor='black', alpha=0.8)axes[1].set_xlabel('Cluster', fontsize=12)axes[1].set_ylabel('Number of Individuals', fontsize=12)axes[1].set_title('Cluster Size Distribution (Bar Chart)', fontsize=14, fontweight='bold')axes[1].set_xticks(cluster_sizes.index)for bar, count in zip(bars, cluster_sizes.values):    height = bar.get_height()    axes[1].annotate(f'{count}\n({count/len(data)*100:.1f}%)',                    xy=(bar.get_x() + bar.get_width() / 2, height),                    xytext=(0, 3), textcoords="offset points",                    ha='center', va='bottom', fontsize=10)plt.tight_layout()plt.savefig(os.path.join(OUTPUT_DIR, 'figures', 'plots', '11_cluster_distribution.png'), dpi=300, bbox_inches='tight')plt.show()print(f"\n[OK] Figure saved: {os.path.join(OUTPUT_DIR, 'figures', 'plots', '11_cluster_distribution.png')}")

## Phase 17: Probability Uncertainty Visualization

This section visualizes the uncertainty in cluster assignments.


In [ ]:
# PHASE 17: PROBABILITY UNCERTAINTY VISUALIZATIONprint("=" * 70)print("PROBABILITY UNCERTAINTY VISUALIZATION")print("=" * 70)# Get maximum probability for each samplemax_probs = probabilities.max(axis=1)data['max_probability'] = max_probs# Categorize confidencehigh_conf = (max_probs >= 0.8).sum()mod_conf = ((max_probs >= 0.5) & (max_probs < 0.8)).sum()low_conf = (max_probs < 0.5).sum()print(f"\nConfidence Level Summary:")print(f"  High Confidence (>=0.8): {high_conf} ({100*high_conf/len(data):.1f}%)")print(f"  Moderate Confidence (0.5-0.8): {mod_conf} ({100*mod_conf/len(data):.1f}%)")print(f"  Low Confidence (<0.5): {low_conf} ({100*low_conf/len(data):.1f}%)")# Create visualizationsfig, axes = plt.subplots(2, 2, figsize=(14, 10))# Histogramaxes[0, 0].hist(max_probs, bins=50, color='steelblue', edgecolor='black', alpha=0.7)axes[0, 0].axvline(x=0.8, color='green', linestyle='--', linewidth=2, label='High (0.8)')axes[0, 0].axvline(x=0.5, color='orange', linestyle='--', linewidth=2, label='Moderate (0.5)')axes[0, 0].set_xlabel('Maximum Assignment Probability', fontsize=12)axes[0, 0].set_ylabel('Frequency', fontsize=12)axes[0, 0].set_title('Distribution of Assignment Confidence', fontsize=14, fontweight='bold')axes[0, 0].legend()# Pie chartconfidence_counts = [high_conf, mod_conf, low_conf]confidence_labels = [f'High\n(n={high_conf})', f'Moderate\n(n={mod_conf})', f'Low\n(n={low_conf})']colors = ['#2ecc71', '#f39c12', '#e74c3c']axes[0, 1].pie(confidence_counts, labels=confidence_labels, autopct='%1.1f%%',                colors=colors, explode=[0.02, 0.02, 0.05], shadow=True)axes[0, 1].set_title('Confidence Level Distribution', fontsize=14, fontweight='bold')# Box plot by clustercluster_prob_data = [data[data['cluster'] == c]['max_probability'].values for c in sorted(data['cluster'].unique())]bp = axes[1, 0].boxplot(cluster_prob_data, labels=[f'Cluster {c}' for c in sorted(data['cluster'].unique())], patch_artist=True)for patch, color in zip(bp['boxes'], plt.cm.Set2(np.linspace(0, 1, len(cluster_prob_data)))):    patch.set_facecolor(color)axes[1, 0].set_xlabel('Cluster', fontsize=12)axes[1, 0].set_ylabel('Maximum Probability', fontsize=12)axes[1, 0].set_title('Assignment Confidence by Cluster', fontsize=14, fontweight='bold')# Heatmapprob_means = probabilities.mean(axis=0)im = axes[1, 1].imshow(prob_means.T, aspect='auto', cmap='YlOrRd')axes[1, 1].set_xlabel('Sample Index (sorted)', fontsize=12)axes[1, 1].set_ylabel('Cluster', fontsize=12)axes[1, 1].set_title('Cluster Probability Heatmap', fontsize=14, fontweight='bold')axes[1, 1].set_yticks(range(len(prob_means)))axes[1, 1].set_yticklabels([f'Cluster {i}' for i in range(len(prob_means))])plt.colorbar(im, ax=axes[1, 1])plt.tight_layout()plt.savefig(os.path.join(OUTPUT_DIR, 'figures', 'plots', '12_probability_uncertainty.png'), dpi=300, bbox_inches='tight')plt.show()print(f"\n[OK] Figure saved: {os.path.join(OUTPUT_DIR, 'figures', 'plots', '12_probability_uncertainty.png')}")

## Phase 18: Demographics and Cluster Association

This section analyzes the relationship between demographic variables and cluster membership.


In [ ]:
# PHASE 18: DEMOGRAPHICS AND CLUSTER ASSOCIATIONprint("=" * 70)print("DEMOGRAPHICS AND CLUSTER ASSOCIATION")print("=" * 70)from scipy.stats import chi2_contingencydemographic_vars = ['gender', 'race/ethnicity', 'age_group']demographic_names = ['Gender', 'Race/Ethnicity', 'Age Group']print("\nChi-Square Tests for Demographics:")print("=" * 70)for var, name in zip(demographic_vars, demographic_names):    if var in data.columns:        contingency_table = pd.crosstab(data[var], data['cluster'])        chi2, p_value, dof, expected = chi2_contingency(contingency_table)        print(f"\n{name}:")        print(f"  Chi-square statistic: {chi2:.4f}")        print(f"  P-value: {p_value:.6f}")        print(f"  Significant: {'Yes' if p_value < 0.05 else 'No'}")# Create visualizationsfig, axes = plt.subplots(2, 2, figsize=(14, 10))if 'gender' in data.columns:    gender_cluster = pd.crosstab(data['gender'], data['cluster'], normalize='index') * 100    gender_cluster.plot(kind='bar', ax=axes[0, 0], colormap='Set2', edgecolor='black')    axes[0, 0].set_title('Cluster Distribution by Gender', fontsize=14, fontweight='bold')    axes[0, 0].tick_params(axis='x', rotation=0)if 'age_group' in data.columns:    age_cluster = pd.crosstab(data['age_group'], data['cluster'], normalize='index') * 100    age_cluster.plot(kind='bar', ax=axes[0, 1], colormap='Set2', edgecolor='black')    axes[0, 1].set_title('Cluster Distribution by Age Group', fontsize=14, fontweight='bold')    axes[0, 1].tick_params(axis='x', rotation=45)if 'race/ethnicity' in data.columns:    race_cluster = pd.crosstab(data['cluster'], data['race/ethnicity'], normalize='index') * 100    race_cluster.plot(kind='barh', stacked=True, ax=axes[1, 0], colormap='Set2', edgecolor='black')    axes[1, 0].set_title('Race/Ethnicity by Cluster', fontsize=14, fontweight='bold')for cluster in sorted(data['cluster'].unique()):    cluster_ages = data[data['cluster'] == cluster]['age']    axes[1, 1].hist(cluster_ages, bins=20, alpha=0.5, label=f'Cluster {cluster}', edgecolor='black')axes[1, 1].set_title('Age Distribution by Cluster', fontsize=14, fontweight='bold')axes[1, 1].set_xlabel('Age (years)', fontsize=12)axes[1, 1].legend()plt.tight_layout()plt.savefig(os.path.join(OUTPUT_DIR, 'figures', 'plots', '13_demographic_analysis.png'), dpi=300, bbox_inches='tight')plt.show()print(f"\n[OK] Figure saved: {os.path.join(OUTPUT_DIR, 'figures', 'plots', '13_demographic_analysis.png')}")

## Phase 19: Final Summary and Export

This section provides a comprehensive summary of the GMM clustering analysis and exports all results.


In [ ]:
# PHASE 19: FINAL SUMMARY AND EXPORTprint("=" * 70)print("FINAL SUMMARY AND EXPORT")print("=" * 70)# Complete summary statisticsprint("\n" + "-" * 70)print("COMPLETE PROJECT SUMMARY")print("-" * 70)print(f"DATASET CHARACTERISTICS:")print(f"  Total Samples: {len(data):,}")print(f"  Features Used: {len(FEATURE_COLUMNS)}")print(f"\nMODEL CONFIGURATION:")print(f"  Algorithm: Gaussian Mixture Models (GMM)")print(f"  Number of Components: {n_clusters}")print(f"\nMODEL PERFORMANCE:")print(f"  BIC Score: {full_eval['bic']:.2f}")print(f"  AIC Score: {full_eval['aic']:.2f}")print(f"  Silhouette Score: {full_eval['silhouette']:.4f}")print(f"\nCLUSTER SUMMARY:")for c in range(n_clusters):    cluster_subset = data[data['cluster'] == c]    print(f"  Cluster {c} ({len(cluster_subset):,} individuals, {100*len(cluster_subset)/len(data):.1f}%):")    print(f"    Mean Age: {cluster_subset['age'].mean():.1f} years")    print(f"    Mean BMI: {cluster_subset['bmi'].mean():.1f}")print(f"\nUNCERTAINTY ANALYSIS:")print(f"  High Confidence (>=0.8): {high_conf} ({100*high_conf/len(data):.1f}%)")print(f"  Moderate Confidence: {mod_conf} ({100*mod_conf/len(data):.1f}%)")print(f"  Low Confidence (<0.5): {low_conf} ({100*low_conf/len(data):.1f}%)")# Export all resultsprint("\n[INFO] Exporting results...")# Save complete dataset with cluster assignmentsexport_df = data.copy()export_df.to_csv(os.path.join(OUTPUT_DIR, 'predictions', 'complete_cluster_assignments.csv'), index=False)print("  [OK] Complete cluster assignments saved")# Save cluster profilescluster_profiles_export = cluster_profiles.copy()cluster_profiles_export['n_individuals'] = cluster_sizes.valuescluster_profiles_export['proportion'] = cluster_proportions.valuescluster_profiles_export.to_csv(os.path.join(OUTPUT_DIR, 'cluster_profiles', 'detailed_cluster_profiles.csv'))print("  [OK] Detailed cluster profiles saved")# Save grid search resultsgrid_results_df = pd.DataFrame({    'n_components': list(n_components_range),    'bic': bic_scores,    'aic': aic_scores})grid_results_df.to_csv(os.path.join(OUTPUT_DIR, 'model_selection', 'grid_search_results.csv'), index=False)print("  [OK] Grid search results saved")# Save probability assignmentsprob_df = pd.DataFrame(probabilities, columns=[f'Cluster_{i}_Prob' for i in range(n_clusters)])prob_df['Predicted_Cluster'] = data['cluster'].valuesprob_df['Max_Probability'] = data['max_probability'].valuesprob_df.to_csv(os.path.join(OUTPUT_DIR, 'predictions', 'cluster_probabilities.csv'), index=False)print("  [OK] Cluster probabilities saved")print("\n" + "=" * 70)print("ANALYSIS COMPLETE")print("=" * 70)print(f"\nAll results saved to: {OUTPUT_DIR}")

## Phase 20: References

1. McLachlan, G. J., & Peel, D. (2000). Finite Mixture Models. John Wiley & Sons.

2. Schwarz, G. (1978). Estimating the dimension of a model. The Annals of Statistics.

3. Pedregosa, F., et al. (2011). Scikit-learn: Machine Learning in Python. JMLR 12, 2825-2830.
